# Data wrangling

In [1]:
import kagglehub
import pandas as pd
import numpy as np
from kagglehub import KaggleDatasetAdapter

encoding_type='dummies'

## 1. Download disaster data from Kaggle

In [2]:
# Load a DataFrame with a specific version of a CSV
raw_data_df=kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    'headsortails/us-natural-disaster-declarations',
    'us_disaster_declarations.csv',
)

In [3]:
raw_data_df.head().transpose()

,0,1,2,3,4
fema_declaration_string,DR-1-GA,DR-2-TX,DR-3-LA,DR-4-MI,DR-5-MT
disaster_number,1,2,3,4,5
state,GA,TX,LA,MI,MT
declaration_type,DR,DR,DR,DR,DR
declaration_date,1953-05-02T00:00:00Z,1953-05-15T00:00:00Z,1953-05-29T00:00:00Z,1953-06-02T00:00:00Z,1953-06-06T00:00:00Z
fy_declared,1953,1953,1953,1953,1953
incident_type,Tornado,Tornado,Flood,Tornado,Flood
declaration_title,Tornado,Tornado & Heavy Rainfall,Flood,Tornado,Floods
ih_program_declared,0,0,0,0,0
ia_program_declared,1,1,1,1,1


In [4]:
raw_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64092 entries, 0 to 64091
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   fema_declaration_string     64092 non-null  object
 1   disaster_number             64092 non-null  int64 
 2   state                       64092 non-null  object
 3   declaration_type            64092 non-null  object
 4   declaration_date            64092 non-null  object
 5   fy_declared                 64092 non-null  int64 
 6   incident_type               64092 non-null  object
 7   declaration_title           64092 non-null  object
 8   ih_program_declared         64092 non-null  int64 
 9   ia_program_declared         64092 non-null  int64 
 10  pa_program_declared         64092 non-null  int64 
 11  hm_program_declared         64092 non-null  int64 
 12  incident_begin_date         64092 non-null  object
 13  incident_end_date           55682 non-null  ob

In [5]:
raw_data_df.to_parquet('../data/raw_disaster_data.parquet')

## 2. Download FIPS code database from census.gov

In [6]:
fips_df=pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt', sep='|')
fips_df.head()

,STATE,STATEFP,COUNTYFP,COUNTYNS,COUNTYNAME,CLASSFP,FUNCSTAT
0,AL,1,1,161526,Autauga County,H1,A
1,AL,1,3,161527,Baldwin County,H1,A
2,AL,1,5,161528,Barbour County,H1,A
3,AL,1,7,161529,Bibb County,H1,A
4,AL,1,9,161530,Blount County,H1,A


In [7]:
# Save the raw data
fips_df.to_parquet('../data/fips_codes.parquet')

## 3. Add county name based on FIPS code

To add the county name, we need to concatenate the `STATEFP` and `COUNTYFP` columns in the census data, then use that string to translate between the `fips` column in the disaster data and the `COUNTYNAME` column in the census data. Let's put the logic in a function for easy refactoring.

In [8]:
def decode_fips(fips_df: pd.DataFrame, raw_data_df: pd.DataFrame) -> pd.DataFrame:
    '''Takes census.gov FIPS dataframe and disaster dataframe, adds human readable 
    county column to disaster data using the census data as look-up table returns 
    updated disaster dataframe'''

    # First extract the state and county FIPS codes
    state_fp=fips_df['STATEFP'].to_list()
    county_fp=fips_df['COUNTYFP'].to_list()

    # Left zero pad state and county FIPS codes to two and three digits respectively
    state_fp=[str(n).zfill(2) for n in state_fp]
    county_fp=[str(n).zfill(3) for n in county_fp]

    # Concatenate the state and county codes to get the full FIPS code
    fips=[i+j for i,j in zip(state_fp, county_fp)]

    # Make a dictionary to translate FIPS county codes to county names
    fips_lookup=dict(zip(fips, fips_df['COUNTYNAME']))

    # Add a new column to the raw disaster data containing the FIPS code to be translated
    data_df=raw_data_df.copy()
    data_df['county_name']=raw_data_df['fips'].apply(str)

    # Translate the column values from FIPS to county name, using 'Unknown' for County if we don't
    # have the FIPS code in our dict
    data_df['county_name']=data_df['county_name'].map(fips_lookup).fillna('Unknown')

    return data_df

In [9]:
# Do the decoding
data_df=decode_fips(fips_df, raw_data_df)

## 4. Save the data

In [10]:
# Take a look at what we have
data_df.tail().transpose()

,64087,64088,64089,64090,64091
fema_declaration_string,DR-4696-ME,DR-4697-MS,DR-4697-MS,DR-4697-MS,DR-4697-MS
disaster_number,4696,4697,4697,4697,4697
state,ME,MS,MS,MS,MS
declaration_type,DR,DR,DR,DR,DR
declaration_date,2023-03-22T00:00:00Z,2023-03-26T00:00:00Z,2023-03-26T00:00:00Z,2023-03-26T00:00:00Z,2023-03-26T00:00:00Z
fy_declared,2023,2023,2023,2023,2023
incident_type,Severe Storm,Severe Storm,Severe Storm,Severe Storm,Severe Storm
declaration_title,Severe Storm And Flooding,"Severe Storms, Straight-Line Winds, And Tornadoes","Severe Storms, Straight-Line Winds, And Tornadoes","Severe Storms, Straight-Line Winds, And Tornadoes","Severe Storms, Straight-Line Winds, And Tornadoes"
ih_program_declared,0,1,1,1,1
ia_program_declared,0,0,0,0,0


In [11]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64092 entries, 0 to 64091
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   fema_declaration_string     64092 non-null  object
 1   disaster_number             64092 non-null  int64 
 2   state                       64092 non-null  object
 3   declaration_type            64092 non-null  object
 4   declaration_date            64092 non-null  object
 5   fy_declared                 64092 non-null  int64 
 6   incident_type               64092 non-null  object
 7   declaration_title           64092 non-null  object
 8   ih_program_declared         64092 non-null  int64 
 9   ia_program_declared         64092 non-null  int64 
 10  pa_program_declared         64092 non-null  int64 
 11  hm_program_declared         64092 non-null  int64 
 12  incident_begin_date         64092 non-null  object
 13  incident_end_date           55682 non-null  ob

In [12]:
# Save the data
data_df.to_parquet('../data/master_disaster_data.parquet')

## 5. Data shape formatting

Now the real work begins - we need to set this dataset up for modeling as a multi-label time-series prediction problem. To make our lives a little easier, we will treat each county as an independent sample (this is likely not strictly true, as especially weather based disasters may tend to co-occur geographically). Doing so increases the number of observations and simplifies the input. Each individual time point will be a vector of dummy encoded disasters, including one feature for 'no-disaster'. This will give is n different time series for the n different counties. Care will need to be taken when splitting/sampling and generating batches not to cross between the time series from different counties.

Let's dig in!

### 5.1. Feature selection

In [13]:
# Get only the features we are going to work with into a new dataframe
working_df=data_df[['incident_begin_date','state','incident_type']].copy()

# Convert 'incident_begin_date' to month and year columns and set as index
working_df['incident_begin_date']=pd.to_datetime(working_df['incident_begin_date'])

# Fix the index
working_df.reset_index(inplace=True, drop=True)
working_df.head()

,incident_begin_date,state,incident_type
0,1953-05-02 00:00:00+00:00,GA,Tornado
1,1953-05-15 00:00:00+00:00,TX,Tornado
2,1953-05-29 00:00:00+00:00,LA,Flood
3,1953-06-02 00:00:00+00:00,MI,Tornado
4,1953-06-06 00:00:00+00:00,MT,Flood


In [14]:
working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64092 entries, 0 to 64091
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   incident_begin_date  64092 non-null  datetime64[ns, UTC]
 1   state                64092 non-null  object             
 2   incident_type        64092 non-null  object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 1.5+ MB


### 5.2. Disaster encoding

In [15]:
# Define target disaster types for summation and prediction
target_disasters=['Severe Storm', 'Hurricane', 'Flood', 'Coastal Storm']

In [16]:
if encoding_type == 'dummies':

    # Encode the disaster types to dummies
    disaster_dummies_df=pd.get_dummies(working_df['incident_type'], dtype=int)

    # Sum our target disasters
    incidents=disaster_dummies_df[target_disasters].sum(axis=1)

    # Add the disaster sums to the data frame as a new feature
    working_df['incidents']=incidents

    # Drop the string incident_type column
    working_df.drop('incident_type', axis=1, inplace=True)

#     disaster_drops=['Biological', 'Chemical', 'Earthquake', 'Fishing Losses', 'Human Cause', 'Other', 'Terrorist', 'Toxic Substances', 'Volcanic Eruption', 'Tropical Storm', 'Tsunami', 'Dam/Levee Break', 'Drought', 'Mud/Landslide']
#     disaster_dummies.drop(disaster_drops, axis=1, inplace=True)

#     # We also have 4 disaster types that are winter weather related, let's combine those
#     disaster_dummies['Winter weather']=disaster_dummies['Severe Ice Storm'] + disaster_dummies['Snowstorm'] + disaster_dummies['Freezing'] + disaster_dummies['Winter Storm']
#     disaster_dummies.drop(['Severe Ice Storm','Snowstorm','Freezing','Winter Storm'], axis=1, inplace=True)
    
#     # Combine the disaster dummies and the original dataframe and remove the 'incident_type' column
#     working_df=pd.concat([working_df.reset_index(drop=True), disaster_dummies.reset_index(drop=True)], axis=1)
#     working_df.drop('incident_type', axis=1, inplace=True)

# if encoding_type == 'multiclass':

#     excluded_incident_types=['Other', 'Volcanic Eruption', 'Earthquake', 'Toxic Substances', 'Fishing Losses', 'Human Cause', 'Terrorist', 'Chemical', 'Biological'] 
#     working_df['incident_type']=working_df['incident_type'].apply(lambda x: x if x not in excluded_incident_types else pd.NA)
#     working_df.dropna(inplace=True)
#     incident_types=working_df['incident_type'].unique().tolist()

#     incidents={}

#     for i, incident in enumerate(incident_types):
#         incidents[incident]=str(i)

#     working_df.replace(incidents, inplace=True)

working_df.head(20)
print(f"Have {working_df['incidents'].sum()} total disaster incidents")

Have 41865 total disaster incidents


Now, we need to regularize the time series to a frequency of months across the span of years within each state.

### 5.3. Time series regularization

In [20]:
def sum_months(group: pd.DataFrame) -> pd.DataFrame:
    '''Takes a yearly groupby object and sums features over months'''

    group=group.resample('ME').sum()

    return group


def resample_months(group: pd.DataFrame) -> pd.DataFrame:
    '''Takes working dataframe and resamples frequency to months.
    Returns updated dataframe'''

    # Set 'incident_begin_date' as datetime axis
    group=group.set_index('incident_begin_date')

    # Sum the disasters in each month by year. This removes duplicates where
    # there was more than one disaster in a month.
    group=group.groupby(group.index.year, group_keys=False).apply(sum_months)

    # Resample to monthly frequency
    group=group.resample('ME').asfreq()

    # Fill missing values with 0
    group=group.fillna(0)

    # Convert everything to int
    group=group.astype(int)

    # Reset the index, preserving the `incident_begin_date`
    group.reset_index(inplace=True, drop=False)

    return group

# Do the resampling
resampled_working_df=working_df.groupby('state', group_keys=True).apply(resample_months, include_groups=False)
resampled_working_df.head()

incident_begin_date  incidents
state                                       
AK    0 1953-10-31 00:00:00+00:00          0
      1 1953-11-30 00:00:00+00:00          0
      2 1953-12-31 00:00:00+00:00          0
      3 1954-01-31 00:00:00+00:00          0
      4 1954-02-28 00:00:00+00:00          0

In [21]:
resampled_working_df.info()
print(f"\nHave {working_df['incidents'].sum()} total disaster incidents")

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 43639 entries, ('AK', np.int64(0)) to ('WY', np.int64(686))
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   incident_begin_date  43639 non-null  datetime64[ns, UTC]
 1   incidents            43639 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(1)
memory usage: 851.1+ KB

Have 41865 total disaster incidents


### 5.4. Add 'no incident' feature

In [22]:
# no_incident=[]

# for i, row in resampled_working_df.iterrows():

#     if sum(row.iloc[1:-1]) == 0:
#         no_incident.append(1)

#     else:
#         no_incident.append(0)

# resampled_working_df['No incident']=no_incident
# resampled_working_df.head(20)

### 5.5. Clean up and finalize features

In [23]:
# resampled_working_df.iloc[:, 1:-1]=resampled_working_df.iloc[:, 1:-1].clip(0, 1)

In [24]:
# Clean up the index
resampled_working_df.reset_index(inplace=True)
resampled_working_df.drop('level_1', axis=1, inplace=True)

In [25]:
# # Fix the FIPS data type
# resampled_working_df['fips']=resampled_working_df['fips'].astype(np.int32)

In [26]:
# Extract month and year from 'incident_begin_date' and drop
resampled_working_df['year']=resampled_working_df['incident_begin_date'].dt.year.astype(int)
resampled_working_df['month']=resampled_working_df['incident_begin_date'].dt.month.astype(int)
resampled_working_df.drop('incident_begin_date', axis=1, inplace=True)
resampled_working_df.head()

,state,incidents,year,month
0,AK,0,1953,10
1,AK,0,1953,11
2,AK,0,1953,12
3,AK,0,1954,1
4,AK,0,1954,2


In [27]:
resampled_working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43639 entries, 0 to 43638
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   state      43639 non-null  object
 1   incidents  43639 non-null  int64 
 2   year       43639 non-null  int64 
 3   month      43639 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.3+ MB


In [28]:
resampled_working_df.describe()

,incidents,year,month
count,43639.000000,43639.000000,43639.000000
mean,0.959348,1989.346112,6.504709
std,7.790390,18.875705,3.450543
min,0.000000,1953.000000,1.000000
25%,0.000000,1973.000000,4.000000
50%,0.000000,1990.000000,7.000000
75%,0.000000,2006.000000,10.000000
max,509.000000,2023.000000,12.000000


### 5.5. Sanity check incident feature vector sum

In [39]:
# # Check and make sure that the incident feature vectors make sense
# row_sum_counts={
#     '0':0,
#     '1':0,
#     '>1':0
# }

# for i, row in resampled_working_df.iterrows():

#     if sum(row.iloc[1:-2]) == 0:
#         row_sum_counts['0']+=1

#     elif sum(row.iloc[1:-2]) == 1:
#         row_sum_counts['1']+=1

#     else:
#         row_sum_counts['>1']+=1

# print(row_sum_counts)

{'0': 0, '1': 43324, '>1': 315}


In [29]:
print(f"\nHave {working_df['incidents'].sum()} total disaster incidents")


Have 41865 total disaster incidents


Note: switching to state as location aggregation level, dropping and combining disaster types and resampling with a frequency of months made our dataset size problem go away. Nice.

## 6. Save the resampled data

### 6.1. Complete dataset

In [30]:
# Save the complete dataset
resampled_working_df.to_parquet('../data/resampled_disaster_data_all.parquet')

### 6.2. Most recent 5 years

In [31]:
# Also save a subset from the last 5 years
sample_resampled_working_df=resampled_working_df[resampled_working_df['year'] >= 1998]
sample_resampled_working_df.reset_index(inplace=True, drop=True)
sample_resampled_working_df.to_parquet('../data/resampled_disaster_data_1998-current.parquet')
sample_resampled_working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16384 entries, 0 to 16383
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   state      16384 non-null  object
 1   incidents  16384 non-null  int64 
 2   year       16384 non-null  int64 
 3   month      16384 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 512.1+ KB
